In [1]:
import pandas as pd
import datetime

from capstone_modules import Production_Data, Temperature_Data, ICO_Composite_Indicator

from statsmodels.tsa.seasonal import seasonal_decompose

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Import data

In [5]:
production = Production_Data()
temperature = Temperature_Data()
ICO_composite_indicator = ICO_Composite_Indicator()
